## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 하이브 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [21]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
item_code = "352820"
item_name = "하이브"

#item_code = "326030"
#item_name = "SK바이오팜"

# 종목 URL 만들기
url = "https://finance.naver.com/item/sise_day.nhn?code=352820&page=1"
url

'https://finance.naver.com/item/sise_day.nhn?code=352820&page=1'

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)


url이 안나오는 이유는 무분별한 접근을 막기위해 header를 숨겨 놨기 때문이다 

f12 ->  가지고 오려는 url에 있는 테이블 항목 아무거나 클릭 -> 네트워크 ->headers -> 쿼리스트링 항목 위에있는 user-agent내용 복사

{"key":"value"}형태로 가지고온 user-agent를 초기화 해주고 url의 headers에 초기화 하면 된다.

headers = {"user-agent":"복사주소"}  -> 이값을 url의 headers옵션에 초기화해주면 된다.

In [31]:
import requests

In [39]:
headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}
headers

{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}

In [43]:
type(headers)

dict

In [40]:
response = requests.get(url,headers=headers)
response

<Response [200]>

In [44]:
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

In [45]:
from bs4 import BeautifulSoup as bs

In [46]:
bs

bs4.BeautifulSoup

In [49]:
html = bs(response.text,"lxml")
html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210624194556/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="

In [53]:
temp=html.select("table")
len(temp)

2

In [58]:
temp

[<table cellspacing="0" class="type2">
 <tr>
 <th>날짜</th>
 <th>종가</th>
 <th>전일비</th>
 <th>시가</th>
 <th>고가</th>
 <th>저가</th>
 <th>거래량</th>
 </tr>
 <tr>
 <td colspan="7" height="8"></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2021.06.28</span></td>
 <td class="num"><span class="tah p11">312,000</span></td>
 <td class="num">
 <img alt="하락" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_down.gif" style="margin-right:4px;" width="7"/><span class="tah p11 nv01">
 				2,000
 				</span>
 </td>
 <td class="num"><span class="tah p11">310,500</span></td>
 <td class="num"><span class="tah p11">317,000</span></td>
 <td class="num"><span class="tah p11">307,500</span></td>
 <td class="num"><span class="tah p11">262,112</span></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2021.06.25</span></td>
 <td class="num"><span clas

## pandas 코드 한 줄로 데이터 수집하기

In [59]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
# table

table=pd.read_html(str(temp))
len(table)

2

In [22]:
print(url)

https://finance.naver.com/item/sise_day.nhn?code=352820&page=1


In [62]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
df_temp = table[0]
df_temp

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.06.28,312000.0,2000.0,310500.0,317000.0,307500.0,262112.0
2,2021.06.25,314000.0,8000.0,326000.0,326500.0,312000.0,354850.0
3,2021.06.24,322000.0,500.0,326000.0,329500.0,317500.0,375522.0
4,2021.06.23,321500.0,3000.0,334500.0,337000.0,318500.0,687701.0
5,2021.06.22,324500.0,16500.0,311000.0,324500.0,311000.0,607966.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021.06.21,308000.0,5000.0,313000.0,321000.0,299500.0,556480.0


In [65]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
df_day = df_temp.dropna()
df_day

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.06.28,312000.0,2000.0,310500.0,317000.0,307500.0,262112.0
2,2021.06.25,314000.0,8000.0,326000.0,326500.0,312000.0,354850.0
3,2021.06.24,322000.0,500.0,326000.0,329500.0,317500.0,375522.0
4,2021.06.23,321500.0,3000.0,334500.0,337000.0,318500.0,687701.0
5,2021.06.22,324500.0,16500.0,311000.0,324500.0,311000.0,607966.0
9,2021.06.21,308000.0,5000.0,313000.0,321000.0,299500.0,556480.0
10,2021.06.18,313000.0,16000.0,298500.0,326500.0,297500.0,1630734.0
11,2021.06.17,297000.0,5000.0,292000.0,301000.0,289000.0,567305.0
12,2021.06.16,292000.0,3000.0,290500.0,298000.0,287000.0,478261.0
13,2021.06.15,289000.0,7000.0,285000.0,293000.0,279500.0,461455.0


## 페이지별 데이터 수집 함수 만들기

In [78]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    url= f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}
    response = requests.get(url,headers=headers)
    html = bs(response.text,"lxml")
    temp = html.select("table")
    table=pd.read_html(str(temp))
    df_day=table[0].dropna()
    return df_day


In [82]:
# 함수가 잘 만들어졌는지 확인하기
page_no=3
item_code = "005930"
get_day_list(item_code,page_no)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.05.31,80500.0,400.0,80300.0,80600.0,79600.0,13321324.0
2,2021.05.28,80100.0,500.0,79800.0,80400.0,79400.0,12360199.0
3,2021.05.27,79600.0,200.0,80000.0,80000.0,79100.0,23198510.0
4,2021.05.26,79800.0,100.0,80400.0,80500.0,79700.0,11984359.0
5,2021.05.25,79900.0,200.0,80000.0,80400.0,79800.0,13628942.0
9,2021.05.24,79700.0,400.0,80100.0,80400.0,79500.0,13398666.0
10,2021.05.21,80100.0,600.0,80100.0,81500.0,79800.0,20961714.0
11,2021.05.20,79500.0,100.0,79400.0,79700.0,79100.0,16541828.0
12,2021.05.18,79600.0,0.0,79600.0,80000.0,79100.0,17494407.0
13,2021.05.17,79600.0,500.0,80800.0,80900.0,79200.0,17740855.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [94]:
import numpy as np
random_sleep = np.random.uniform(0,1)
random_sleep

0.4068344682667483

In [95]:
import time
# web page 시작번호
page_no = 1
start_page = 1
end_page = 10
# 데이터를 저장할 빈 변수 선언
item_list = []
for page_no in range(start_page, end_page+1):
    df_temp_day= get_day_list(item_code,page_no)
    item_list.append(df_temp_day)
    random_sleep = np.random.uniform(0,1)
    time.sleep(random_sleep)

In [96]:
len(item_list)

10

In [97]:
item_list

[            날짜       종가     전일비       시가       고가       저가         거래량
 1   2021.06.28  81900.0   300.0  81700.0  82000.0  81600.0  11315720.0
 2   2021.06.25  81600.0   400.0  81500.0  81900.0  81200.0  13481405.0
 3   2021.06.24  81200.0  1100.0  80400.0  81400.0  80100.0  18771080.0
 4   2021.06.23  80100.0   100.0  80500.0  80600.0  79900.0  13856548.0
 5   2021.06.22  80000.0   100.0  80200.0  80300.0  79900.0  11773365.0
 9   2021.06.21  79900.0   600.0  79700.0  80000.0  79600.0  16063340.0
 10  2021.06.18  80500.0   400.0  81100.0  81100.0  80500.0  14916721.0
 11  2021.06.17  80900.0   900.0  81100.0  81300.0  80700.0  14007385.0
 12  2021.06.16  81800.0   900.0  81500.0  81900.0  81100.0  14999855.0
 13  2021.06.15  80900.0   400.0  80900.0  81200.0  80600.0  10075685.0,
             날짜       종가     전일비       시가       고가       저가         거래량
 1   2021.06.14  80500.0   500.0  80800.0  80900.0  80500.0  10550078.0
 2   2021.06.11  81000.0     0.0  81200.0  81700.0  80700.0  14

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [106]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
a = pd.concat(item_list)
a

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.06.28,81900.0,300.0,81700.0,82000.0,81600.0,11315720.0
2,2021.06.25,81600.0,400.0,81500.0,81900.0,81200.0,13481405.0
3,2021.06.24,81200.0,1100.0,80400.0,81400.0,80100.0,18771080.0
4,2021.06.23,80100.0,100.0,80500.0,80600.0,79900.0,13856548.0
5,2021.06.22,80000.0,100.0,80200.0,80300.0,79900.0,11773365.0
...,...,...,...,...,...,...,...
9,2021.02.08,83000.0,500.0,83800.0,84200.0,83000.0,15338765.0
10,2021.02.05,83500.0,1000.0,83100.0,84000.0,82500.0,18036835.0
11,2021.02.04,82500.0,2100.0,83500.0,83800.0,82100.0,24171688.0
12,2021.02.03,84600.0,200.0,84800.0,85400.0,83400.0,22112205.0


<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [108]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
a.head()

,날짜,종가,전일비,시가,고가,저가,거래량
9,2021.02.08,83000.0,500.0,83800.0,84200.0,83000.0,15338765.0
10,2021.02.05,83500.0,1000.0,83100.0,84000.0,82500.0,18036835.0
11,2021.02.04,82500.0,2100.0,83500.0,83800.0,82100.0,24171688.0
12,2021.02.03,84600.0,200.0,84800.0,85400.0,83400.0,22112205.0
13,2021.02.02,84400.0,1400.0,84100.0,86400.0,83700.0,26302077.0


In [109]:
a.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
9,2021.02.08,83000.0,500.0,83800.0,84200.0,83000.0,15338765.0
10,2021.02.05,83500.0,1000.0,83100.0,84000.0,82500.0,18036835.0
11,2021.02.04,82500.0,2100.0,83500.0,83800.0,82100.0,24171688.0
12,2021.02.03,84600.0,200.0,84800.0,85400.0,83400.0,22112205.0
13,2021.02.02,84400.0,1400.0,84100.0,86400.0,83700.0,26302077.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [ ]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
# df["종목코드"]
# df["종목명"]

## 컬럼 순서 변경하기

In [ ]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.


In [115]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']

## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [ ]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인


## 기술통계값 구하기

In [ ]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [ ]:
# 거래량의 과학적 기수법 읽기


## 최근 날짜 구해서 파일명 만들기

In [ ]:
# 날짜 column의 첫 row값 확인
# date

In [ ]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
# file_name


## 파일로 저장하기

In [ ]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.


In [ ]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.


## 전체 과정을 하나의 함수로 만들기

In [ ]:
# def get_item_list(item_code, item_name):
#     """
#     일별 시세를 수집하는 함수
#     """

In [ ]:
# item_code = "352820"
# item_name = "하이브"

item_code = "326030"
item_name = "SK바이오팜"


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [ ]:
# 도움말 보기
# get_item_list?


In [ ]:
# 소스코드 보기
# get_item_list??
